### Variables

In [0]:
ON_COLAB = True
experiment = 'exp00'

In [0]:
if ON_COLAB:
    project_dir = 'drive/My Drive/Colab Notebooks/TextGen'
    local_root = ''
else:
    project_dir = '..'
    local_root = '..'

### Setup and Imports

In [0]:
# Load the "autoreload" extension so that code can change
%load_ext autoreload

# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import os
import sys

In [0]:
if ON_COLAB:
    %tensorflow_version 2.x

In [5]:
if ON_COLAB:# Allow access to src functions from google drive
    from google.colab import drive
    drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
if ON_COLAB:
    # Install extra requirements (src, socialml)

    # src files (added via path, colab doesn't seem to like installing it via pip)
    src_path = os.path.join(project_dir)
    sys.path.append(src_path)

    # socialml
    !pip install git+https://github.com/erees1/socialml.git#egg=socialml

  Cloning https://github.com/erees1/socialml.git to /tmp/pip-install-c3awv3qu/socialml
  Running command git clone -q https://github.com/erees1/socialml.git /tmp/pip-install-c3awv3qu/socialml
  Created wheel for socialml: filename=socialml-0.1-cp36-none-any.whl size=6993 sha256=81559ccb08050ecd2778bd42f7dbff7fe7da15f8c00505cf1d43ad69e7888de4
  Stored in directory: /tmp/pip-ephem-wheel-cache-gz5zitf0/wheels/29/78/7e/cae2ac03d8e7733ab60f482721cb57629c6e5788af4c061cef
Successfully built socialml


In [0]:
from src.models import seq2seq, tf_dataset_loader, train_model
from src.data.word_utils import Vocab
import socialml
from src.data import colab_utils
import sys
import os

### Run training script

In [0]:
# Setup file paths relative to project directory
drive_data_loc = os.path.join(project_dir, 'data/processed')
spec_path = os.path.join(project_dir, 'specs', experiment+'.yaml')
log_dir = os.path.join(project_dir, 'models/logs', experiment)
checkpoint_dir = os.path.join(project_dir, 'models/checkpoints', experiment)
train_script_path = os.path.join(project_dir, 'src/models/train_model.py')

In [0]:
# If on colab copy contents of data/processed from gdrive to local machine (reading directly from gdrive is slow)
force_copy = False
if ON_COLAB:
    if not os.path.exists('data/processed') or force_copy:
        colab_utils.copy_to_local(drive_data_loc, 'data/processed')

# Data files relative to processed data directory
vocab_filepath = os.path.join(local_root, 'data/processed', 'vocab_pp.json')
word2vec_filepath = os.path.join(local_root, 'data/processed', 'short-vectors.bin')
X_path = os.path.join(local_root, 'data/processed/train', 'strings_X.txt')
Y_path = os.path.join(local_root, 'data/processed/train', 'strings_Y.txt')

In [0]:
# Run with python script
#!python $train_script_path $array_path --spec_path $spec_path --log_dir $log_dir --checkpoint_dir $checkpoint_dir --vocab_filepath $vocab_filepath

In [11]:
# Run using import (works better on colab)
train_model.build_and_train(
    X_path,
    Y_path,
    spec_path=spec_path,
    log_dir=log_dir,
    checkpoint_dir=checkpoint_dir,
    vocab_filepath=vocab_filepath,
    word2vec_filepath=word2vec_filepath,
)

spec: {'model_params': {'depth': 2, 'encoder_units': 256, 'embedding_dim': 'None', 'use_embedding_layer': 0, 'batch_size': 64, 'encoder_input_shape': [128, 300], 'decoder_input_shape': [129, 300]}, 'tf_dataset_params': {'batch_size': 64, 'buffer_size': 10000, 'reverse_context': 1}, 'training_params': {'epochs': 10}}


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Loaded word2vec mapping
Created tf.data.Dataset


AttributeError: ignored